# CiC Research : Raw survey data into bronze

This notebook, ingest the survey raw data (Excel format). It creates a local `sqlite` database to store the data.

## Activities in this notebook

- convert excel to json or **sqlite**.
- helper method for sqlite (query, non-query)
- Use LLM to extract specific elements from the survey data -> these values are required for the 'overall pain point' analysis.
- Write to a new table with the extracted values from the raw data together with the LLM extraction data
- Provide query to check the updated table


## Excel / CSV 2 JSON / SQLite

The content provided by `Ipsos` team is an excel file. 
We explored the excel2json and excel2sqlite. We decided to use sqlite as it has better performance and ability to query the data in a well known query language. 

In [ ]:
#r "nuget: ClosedXML, 0.104.2"
#r "nuget: Microsoft.Data.Sqlite, 6.0.0"

using System;
using System.IO;
using System.Linq;
using System.Collections.Generic;
using ClosedXML.Excel;
using System.Text.Json;

using Microsoft.Data.Sqlite;

### 📄 Excel Conversion Utilities

This notebook includes two utility methods for converting Excel files into more accessible formats for further analysis:

---

#### ✅ `ConvertExcelToJsonStream(string excelFilePath, string outputJsonPath)`
This method reads the first worksheet of an Excel file and writes its contents to a JSON file as an array of objects.  
- The first row is assumed to be the header and used as JSON property names.  
- Each subsequent row becomes a JSON object, with cell values converted to strings.  
- Data is written to the output stream efficiently with periodic flushing to manage memory.

**Use case:** Preparing Excel data for JSON-based pipelines, APIs, or lightweight visualization tools.

---

#### ✅ `ConvertExcelToSQLite(string excelFilePath, string sqliteDbPath)`
This method reads an Excel file and stores its contents into a SQLite database table named `SurveyResponses`.  
- The first row defines the column names (all stored as `TEXT`).  
- Empty values and `#NULL!` strings are normalized to `"N/A!"`.  
- Insertions are wrapped in a transaction for better performance.

**Use case:** Loading structured Excel data into a local SQLite DB for querying, filtering, or joining with other data sources.

In [ ]:
public bool ConvertExcelToJsonStream(string excelFilePath, string outputJsonPath)
{
    try
    {
        using var workbook = new XLWorkbook(excelFilePath);
        var worksheet = workbook.Worksheet(1);

        // Read the header row dynamically
        var headerRow = worksheet.FirstRowUsed();
        var headers = headerRow.CellsUsed().Select(c => c.GetString()).ToList();

        // Open the output file stream
        using var fs = new FileStream(outputJsonPath, FileMode.Create, FileAccess.Write, FileShare.None);
        var jsonWriterOptions = new JsonWriterOptions { Indented = true };
        using var writer = new Utf8JsonWriter(fs, jsonWriterOptions);

        writer.WriteStartArray();
        int rowCount = 0;

        // Process each row after the header
        foreach (var row in worksheet.RowsUsed().Skip(1))
        {
            writer.WriteStartObject();
            int colIndex = 0;
            foreach (var cell in row.Cells(1, headers.Count))
            {
                // Convert each cell's value to string (you could add type checking if needed)
                string value = cell.Value.ToString() ?? "";
                writer.WriteString(headers[colIndex], value);
                colIndex++;
            }
            writer.WriteEndObject();
            rowCount++;

            // Flush periodically (every 100 rows in this example) to reduce memory pressure
            if (rowCount % 100 == 0)
            {
                writer.Flush();
            }
        }

        writer.WriteEndArray();
        writer.Flush();
        return true;
    }
    catch (Exception ex)
    {
        Console.WriteLine($"Error during conversion: {ex.Message}");
        return false;
    }
}

In [ ]:
public bool ConvertExcelToSQLite(string excelFilePath, string sqliteDbPath)
{
    try
    {
        // Open the Excel workbook using ClosedXML
        using (var workbook = new XLWorkbook(excelFilePath))
        {
            // Get the first worksheet (adjust if needed)
            var worksheet = workbook.Worksheet(1);

            // Read the header row dynamically
            var headerRow = worksheet.FirstRowUsed();
            var headers = headerRow.CellsUsed().Select(c => c.GetString()).ToList();

            // Open (or create) the SQLite database file
            using (var connection = new SqliteConnection($"Data Source={sqliteDbPath}"))
            {
                connection.Open();

                // Create a table with all columns as TEXT.
                // Use square brackets around column names to handle spaces or special characters.
                var columnsDef = string.Join(", ", headers.Select(h => $"[{h}] TEXT"));
                var createTableSql = $"CREATE TABLE IF NOT EXISTS SurveyResponses ({columnsDef});";
                using (var cmd = new SqliteCommand(createTableSql, connection))
                {
                    cmd.ExecuteNonQuery();
                }

                // Build an INSERT statement with parameters for each column.
                var columnsList = string.Join(", ", headers.Select(h => $"[{h}]"));
                var paramList = string.Join(", ", headers.Select((h, i) => $"@p{i}"));
                var insertSql = $"INSERT INTO SurveyResponses ({columnsList}) VALUES ({paramList});";

                // Wrap the insertion in a transaction for better performance.
                using (var transaction = connection.BeginTransaction())
                using (var insertCmd = new SqliteCommand(insertSql, connection, transaction))
                {
                    // Pre-add the parameters to the command.
                    for (int i = 0; i < headers.Count; i++)
                    {
                        insertCmd.Parameters.Add(new SqliteParameter($"@p{i}", ""));
                    }

                    // Process each row (skip the header row)
                    foreach (var row in worksheet.RowsUsed().Skip(1))
                    {
                        int colIndex = 0;
                        foreach (var cell in row.Cells(1, headers.Count))
                        {
                            // Read the cell value as a string.
                            string value = cell.Value.ToString();

                            // Replace empty strings or "#NULL!" with "N/A!"
                            if (string.IsNullOrEmpty(value) || value == "#NULL!")
                            {
                                value = "N/A!";
                            }
                            insertCmd.Parameters[$"@p{colIndex}"].Value = value;
                            colIndex++;
                        }
                        insertCmd.ExecuteNonQuery();
                    }
                    transaction.Commit();
                }
                connection.Close();
            }
        }
        return true;
    }
    catch (Exception ex)
    {
        Console.WriteLine($"Error during conversion: {ex.Message}");
        return false;
    }
}

### Excel2Json

Using the method to convert the excel to a json file. This needs to be executed only once.


In [ ]:
var excelFilePath = "FY25H1 Raw Data with labels US and UK.xlsx";
var outputJsonPath = "fy25-raw.json";

if ( ConvertExcelToJsonStream(excelFilePath, outputJsonPath))
{
    Console.WriteLine($"Excel file '{excelFilePath}' was successfully converted to JSON and saved to '{outputJsonPath}'");
}
else
{
    Console.WriteLine($"Failed to convert '{excelFilePath}' to JSON");
}

### Excel2Sqlite

Using the method to convert the excel to a sqlite file.

In [ ]:
var excelFilePath = "FY25H1 Raw Data with labels US and UK.xlsx";
var outputJsonPath = "fy25-raw.db";

if ( ConvertExcelToSQLite(excelFilePath, outputJsonPath))
{
    Console.WriteLine($"Excel file '{excelFilePath}' was successfully converted to sqllite and saved to '{outputJsonPath}'");
}
else
{
    Console.WriteLine($"Failed to convert '{excelFilePath}' to sql lite");
}

In [ ]:
// include a local class named VectorMath - this class is used to calculate the cosine similarity between two vectors


#load "VectorMath.cs"
#load "ClassificationNode.cs"
#load "SQLiteHelper.cs"

In [ ]:
var connection = SQLiteHelper.LoadDatabase("fy25-raw.db");
string query = "SELECT * from SurveyResponses WHERE ResponseId = 'R_105HrhmcqZ4Edep';";

var results = SQLiteHelper.ExecuteQuery(connection, query);
// Print the results all the fields

foreach (var row in results)
{
    foreach (var kvp in row)
    {
        Console.WriteLine($"{kvp.Key}: {kvp.Value}");
    }
    Console.WriteLine();
}



## query the data

In [ ]:
var dbhelper = SQLiteHelper.LoadDatabase("../fy25-raw.db");
var query = @"select count(*) from SurveyResponses
              where Q026a_8 = 'Very satisfied';";
var results = SQLiteHelper.ExecuteQuery(dbhelper, query);
foreach (var row in results)
{
    Console.WriteLine(string.Join(", ", row.Select(kvp => $"{kvp.Key}={kvp.Value}")));
}

// query = @"select count(*) from MyView
//          where Q026_19 = 'Very satisfied';";
// results = SQLiteHelper.ExecuteQuery(dbhelper, query);
// foreach (var row in results)
// {
//     Console.WriteLine(string.Join(", ", row.Select(kvp => $"{kvp.Key}={kvp.Value}")));
// }

## create table for the theme

In [ ]:
var connection = SQLiteHelper.LoadDatabase("../fy25-raw.db");

// drop a tableName

var tableName = "OverallPailPoints";
SQLiteHelper.DropTableOrView(connection, tableName, "table");

In [ ]:
var connection = SQLiteHelper.LoadDatabase("../fy25-raw.db");

var tableName = "OverallPailPoints";

string query1 = $@"
    CREATE TABLE IF NOT EXISTS {tableName} AS
    SELECT ResponseId, 
           BrandAssigned, 
           SAM11 as Country, 
           TAXALN, 
           CASE 
               WHEN Q005 LIKE 'Other%' THEN 'Other'
               ELSE Q005
           END AS Q005,
           CASE 
               WHEN Q005 LIKE 'Other%' THEN Q005_996_TEXT
               ELSE ''
           END AS Q005_996_TEXT,
           Q009_,
           Cloud_Usage,
           Q082,
           Q048b,
           CASE 
               WHEN Q089a_2 = '#NULL!' THEN ''
               ELSE 'ISV'
           END AS Q089a_2,
           CASE 
               WHEN Q102a = 'No' OR Q102a = '#NULL!' THEN ''
               ELSE 'Startup'
           END AS Q102a,
           Q024b as Verbatim

           FROM SurveyResponses;";

SQLiteHelper.ExecuteNonQuery(connection, query1);

var query2 = $@"
    SELECT * FROM {tableName};";
var results = SQLiteHelper.ExecuteQuery(connection, query2);

foreach (var row in results.Take(10))
{
    Console.WriteLine(string.Join(", ", row.Select(kvp => $"{kvp.Key}={kvp.Value}")));
}


## Open AI - required for classification 

Using Open AI to analyze verbatim and attribution and create:

- Sentiment analysis
- Keyword extraction
- Model scores
- Themes

In [ ]:
#r "nuget: Azure.AI.OpenAI, 2.1.0"
#r "nuget: Azure.Identity, 1.8.0"
#r "nuget: DotNetEnv, 2.5.0"


using Azure.Identity;
using Azure;

using DotNetEnv;

using System.IO;
using System.Text.Json;
using System.ClientModel;

using Azure.AI.OpenAI;
using Azure.AI.OpenAI.Chat;

using OpenAI.Chat;

using OpenAI.Embeddings;

In [ ]:
string _configurationFile = @"../../../../configuration/.env";
Env.Load(_configurationFile);

string oAiEndpoint = Environment.GetEnvironmentVariable("AOAI_ENDPOINT") 
    ?? "AOAI_ENDPOINT not found";
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("CHATCOMPLETION_DEPLOYMENTNAME") 
    ?? "CHATCOMPLETION_DEPLOYMENTNAME not found";
string embeddingDeploymentName = Environment.GetEnvironmentVariable("EMBEDDING_DEPLOYMENTNAME") ?? "EMBEDDING_DEPLOYMENTNAME not found";
var credential = new DefaultAzureCredential();

// Now create the client using your identity:
AzureOpenAIClient openAIClient = new AzureOpenAIClient(
    new Uri(oAiEndpoint),
    credential
);

Console.WriteLine($"OpenAI Client created with user identity at: {oAiEndpoint}, using deployment: {chatCompletionDeploymentName}");
public static class VectorMath
{
    // If your embeddings are guaranteed to be length 1536, you can fix that in the code.
    // Or you can remove references to VectorDimension and just use vector.Length.

    public const int VectorDimension = 1536;

    public static float Length(float[] vector)
    {
        float sum = 0;
        for (int i = 0; i < VectorDimension; i++)
        {
            sum += vector[i] * vector[i];
        }
        return (float)Math.Sqrt(sum);
    }

    public static float DotProduct(float[] a, float[] b)
    {
        float sum = 0;
        for (int i = 0; i < VectorDimension; i++)
        {
            sum += a[i] * b[i];
        }
        return sum;
    }

    // Standard Cosine Similarity: dot(a, b) / (|a| * |b|)
    public static float CosineSimilarity(float[] a, float[] b)
    {
        float dot = DotProduct(a, b);
        float magA = Length(a);
        float magB = Length(b);

        // Handle potential divide-by-zero if either vector is all zeros
        if (magA < 1e-8f || magB < 1e-8f) return 0f;

        return dot / (magA * magB);
    }
}

async Task<float[]> GetEmbeddingAsync(AzureOpenAIClient _openAIClient,string textToBeVecorized)
{
    // Prepare the embeddings options with the user story\n",
    EmbeddingClient embeddingClient = _openAIClient.GetEmbeddingClient(embeddingDeploymentName);
    ClientResult<OpenAIEmbedding> embeddingResult = await embeddingClient.GenerateEmbeddingAsync(textToBeVecorized);   
    float[] response = embeddingResult?.Value?.ToFloats().ToArray() ?? new float[0];
    return response;
}

In [ ]:
async Task<string> CallOpenAI(AzureOpenAIClient _openAIClient, string prompt, string systemMessage, bool jsonResponse = true)
{
    // Get the chat client (using your deployment or model name)
    ChatClient chatClient = _openAIClient.GetChatClient(chatCompletionDeploymentName);

    ChatCompletionOptions chatComletionOptions = new ChatCompletionOptions(){
        MaxOutputTokenCount = 450,
        Temperature = 0.7f,
        TopP = 1.0f,
        FrequencyPenalty = 0.7f,
        PresencePenalty = 0.7f,

    };

    chatComletionOptions.ResponseFormat = jsonResponse ? ChatResponseFormat.CreateJsonObjectFormat() : ChatResponseFormat.CreateTextFormat();

    // Prepare your messages
    ChatMessage[] messages = new ChatMessage[]
    {
        new SystemChatMessage(systemMessage),
        new UserChatMessage(prompt)
    };

    // Call the chat completions endpoint with parameters directly
    ChatCompletion completions = await chatClient.CompleteChatAsync(        
    messages: messages, 
    options: chatComletionOptions);

    // Get the text from the first completion choice
    // var resp = completions.Content[0];
    
    string result = completions.Content[0].Text;
    return result;
}

### Prompt - reasoning

This is a first attempt on extracting information from the verbatim and attribution. Here we are not providing to the LLM any specific definition of classes, rather allowing the LLM to learn from the data.

In [ ]:
string systemMessage = 
@"You are an AI assistant that analyze survey verbatim and grounding information called attribution.
You are requested to provide several aspects of the survey response, such as the sentiment, keywords and themes and word count.
Model classifications score are based on the keywords.
You always respond in JSON format:
{
    'sentiment': 'positive',
    'keywords': ['product', 'service'],
    'themes': ['customer service', 'product quality'],
    'Model Classification Scores': \""[{'<theme1>': <1.0>}, {'<theme2>': <1.0>}]\""
}";


// Call OpenAI method with sample data - this is how we tested the prompt
var resp = await CallOpenAI(openAIClient, 
    @"Attribution: Google Cloud, US, DATA PRO, Education, , 1,000 to 4,999, Multi-Cloud, 6 years or more, Standard, ISV, Startup, Verbatim: Prices changing, tools changing, things going obsolete/removed",
    systemMessage, false);

// Print the result
Console.WriteLine(resp);

In [ ]:
using System;
using System.Collections.Generic;
using System.Text.Json.Serialization;

public class OpenAIResponse
{
    [JsonPropertyName("sentiment")]
    public string Sentiment { get; set; }
    
    [JsonPropertyName("keywords")]
    public List<string> Keywords { get; set; }
    
    [JsonPropertyName("themes")]
    public List<string> Themes { get; set; }
      
    // Instead of a list with a custom converter, just capture the raw JSON string.
    [JsonPropertyName("Model Classification Scores")]
    public string ModelClassificationScores { get; set; }
}

## Word count Utility

In [ ]:
public static class TextUtilities
{
    /// <summary>
    /// Counts words by splitting on whitespace.
    /// </summary>
    public static int CountWords(string text)
    {
        if (string.IsNullOrWhiteSpace(text))
            return 0;
        // Split by any whitespace and remove empty entries.
        return text.Split((char[])null, StringSplitOptions.RemoveEmptyEntries).Length;
    }
}

In [ ]:
public async Task ProcessAndMapOpenAIResponse_TypedAsync(
    AzureOpenAIClient _openAIClient, 
    string systemMessage, 
    string tableName, 
    int rowLimit = 10)
{
    // Load the database.
    var connection = SQLiteHelper.LoadDatabase("../fy25-raw.db");
    
    // Retrieve a subset of rows from the target table.
    string selectQuery = $"SELECT * FROM {tableName} LIMIT {rowLimit};";
    var results = SQLiteHelper.ExecuteQuery(connection, selectQuery);
    
    // Get the list of existing columns in the table.
    List<string> existingColumns = SQLiteHelper.GetTableColumns(connection, tableName);
    
    foreach (var row in results)
    {
        // Build attribution string from all non-empty fields except "Verbatim".
        var attributionFields = row
            .Where(kvp => !kvp.Key.Equals("Verbatim", StringComparison.OrdinalIgnoreCase)
                          && kvp.Value != null 
                          && !string.IsNullOrWhiteSpace(kvp.Value.ToString()))
            .Select(kvp => kvp.Value.ToString());
        string attribution = string.Join(", ", attributionFields);
    
        // Construct the prompt.
        string userPrompt = $"Attribution: {attribution}, Verbatim: {row["Verbatim"]}";
    
        // Call OpenAI with the prompt.
        // CallOpenAI is assumed to be defined elsewhere and returns a JSON string.
        string resp = await CallOpenAI(_openAIClient, userPrompt, systemMessage, true);
        Console.WriteLine(resp);
        // Parse the JSON response into a strongly typed object.
        OpenAIResponse openAIResponse = null;
        try
        {
            var options = new JsonSerializerOptions { PropertyNameCaseInsensitive = true };
            openAIResponse = JsonSerializer.Deserialize<OpenAIResponse>(resp, options);
        }
        catch (Exception ex)
        {
            Console.WriteLine($"Error parsing JSON response for ResponseId {row["ResponseId"]}: {ex.Message}");
            continue;
        }
    
        // Build a dictionary of the fields you want to update.
        // For example, if you want to update sentiment, keywords, themes, word_count, and model classification scores:
        var updateValues = new Dictionary<string, object>
        {
            { "Sentiment", openAIResponse.Sentiment },
            { "Keywords", openAIResponse.Keywords != null ? string.Join(", ", openAIResponse.Keywords) : "" },
            { "Themes", openAIResponse.Themes != null ? string.Join(", ", openAIResponse.Themes) : "" },
            { "WordCount", TextUtilities.CountWords(row["Verbatim"].ToString()) },
            { "ModelClassificationScores", openAIResponse.ModelClassificationScores  }
        };
    
        // Ensure that each key exists as a column in the table.
        foreach (var key in updateValues.Keys)
        {
            if (!existingColumns.Any(c => c.Equals(key, StringComparison.OrdinalIgnoreCase)))
            {
                string alterQuery = $"ALTER TABLE {tableName} ADD COLUMN [{key}] TEXT;";
                try
                {
                    SQLiteHelper.ExecuteNonQuery(connection, alterQuery);
                    existingColumns.Add(key);
                }
                catch (Exception ex)
                {
                    Console.WriteLine($"Failed to add column {key}: {ex.Message}");
                }
            }
        }
    
        // Update the current row in the database using ResponseId as the unique key.
        SQLiteHelper.UpdateTableRow(connection, tableName, "ResponseId", row["ResponseId"], updateValues);
    
        Console.WriteLine($"Updated ResponseId {row["ResponseId"]} with typed OpenAI response fields.");
    }
}

In [ ]:
// await ProcessAndMapOpenAIResponseAsync(openAIClient, systemMessage, "OverallPailPoints", 1);

await ProcessAndMapOpenAIResponse_TypedAsync(openAIClient, systemMessage, "OverallPailPoints", 1);

In [ ]:
// select from the updated table based on the response id, confirming it got into the db

var connection = SQLiteHelper.LoadDatabase("../fy25-raw.db");

var tableName = "OverallPailPoints";

string query = $@"
    SELECT * FROM {tableName} 
    WHERE ResponseId = 'R_105HrhmcqZ4Edep';";
var results = SQLiteHelper.ExecuteQuery(connection, query);

foreach (var row in results)
{
    foreach (var kvp in row)
    {
        Console.WriteLine($"{kvp.Key}: {kvp.Value}");
    }
    Console.WriteLine();
}

## consider removal

In [ ]:
using System;
using System.Collections.Generic;
using System.Net.Http;
using System.Text.Json;
using System.Threading.Tasks;

public async Task<string> GetCountryFromCoordinatesAsync(double latitude, double longitude)
{
    try
    {
        using HttpClient client = new HttpClient();
        client.DefaultRequestHeaders.UserAgent.ParseAdd("MyAppName/1.0 (yoavdo@gmail.com)");
        // This URL is an example using the free OpenStreetMap Nominatim service.
        // Check usage policies and consider caching for production.
        string url = $"https://nominatim.openstreetmap.org/reverse?format=jsonv2&lat={latitude}&lon={longitude}";
        HttpResponseMessage response = await client.GetAsync(url);
        response.EnsureSuccessStatusCode();
        string jsonResponse = await response.Content.ReadAsStringAsync();
        using JsonDocument doc = JsonDocument.Parse(jsonResponse);
        if (doc.RootElement.TryGetProperty("address", out JsonElement address))
        {
            // Console.WriteLine(address);
            if (address.TryGetProperty("country_code", out JsonElement country_code))
            {
                return country_code.GetString();
            }
        }
    }
    catch (Exception ex)
    {
        Console.WriteLine($"Error reverse geocoding: {ex.Message}");
    }
    return "N/A!";
}

In [ ]:
var connection = SQLiteHelper.LoadDatabase("../fy25-raw.db");

var tableName  = "Locations";

string selectQuery = $@"
    SELECT count(*) FROM {tableName} 
    WHERE Country != '';";
    
var results = SQLiteHelper.ExecuteQuery(connection, selectQuery);

// print the first 10 rows
foreach (var row in results.Take(10))
{
    Console.WriteLine(string.Join(", ", row.Select(kvp => $"{kvp.Key}={kvp.Value}")));
}

### GetCountryFromCoordinatesAsync

Used to translate the longtitude and latitude to country.

In [ ]:
var location = await GetCountryFromCoordinatesAsync(37.7749, -122.4194);
Console.WriteLine(location);

## Data manipulation & exploration

As there are few fields which are calculated, this shows how it can be done. The creation of a new table with the ResponseID is used to allow later join with this data.
It also shows how to do that for other calculations. 

In [ ]:
// select 10 items from the table above
var connection = SQLiteHelper.LoadDatabase("../fy25-raw.db");

var tableName  = "Locations";

string createTableQuery = $@"
    CREATE TABLE IF NOT EXISTS {tableName} AS
    SELECT ResponseId, 
           LocationLatitude, 
           LocationLongitude, 
           '' AS Country
    FROM SurveyResponses;
";
SQLiteHelper.ExecuteNonQuery(connection, createTableQuery);

// update only 10 items (the api might throttle us)
string selectQuery = $@"
    SELECT * FROM {tableName} 
    WHERE Country = ''
    LIMIT 10;";
var results = SQLiteHelper.ExecuteQuery(connection, selectQuery);

foreach (var row in results)
{
    double latitude = Convert.ToDouble(row["LocationLatitude"]);
    double longitude = Convert.ToDouble(row["LocationLongitude"]);
    Console.WriteLine($"Latitude: {latitude}, Longitude: {longitude}");
    string country = await GetCountryFromCoordinatesAsync(latitude, longitude);
    
    // Update the in-memory dictionary (for display, if needed)
    row["Country"] = country.ToUpper();
    Console.WriteLine($"Country: {country.ToUpper()}");
    
    // Update the corresponding row in the database using ResponseId as the key.
    var updatedValues = new Dictionary<string, object>
    {
        { "Country", country.ToUpper() }
    };
    SQLiteHelper.UpdateTableRow(connection, tableName, "ResponseId", row["ResponseId"], updatedValues);
}

Console.WriteLine("Rows updated successfully.");